# Bike Sharing Competition
### by: Rama Salahat
***


in this notebook we're using data from https://www.kaggle.com/c/bike-sharing-demand/overview
that precisely tracks hourly bike rental data spanning two years for the first 19 days of each month. <br><br>
We're going to do an EDA and feature engineer the data to better fit the following models: 
- simple linear regression
- ridge regression
- k-nn
- decision tree

and try to predict future values for the total count of bikes rented during each hour 


<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/3948/media/bikes.png" alt="Bike Sharing Competition" />

## Table of Contents
- [**Ground Truths About Data**](#facts)
- [**EDA and Feature Engineering**](#EDA)
    - [Importing packages and reading the data](#import)
    - [Log Transform](#log)
    - [Distributions of the bookings according to the features](#features)
    - [Distribution of booking by Hours and Day-of-Week Together](#dayhour)
    - [Weather-Related values Collerence](#weather)
    - [Is the workingday atribute enough?](#workingday)
    - [Hot-Encoding the Weather and Season attributes](#hot)
    - [Dropping Useless Columns](#drop)
- [**Machine Learning Models**](#ML)
    - [Linear Regression](#linear)
    - [Linear Regression-polynomial features](#linearp)
    - [Ridge](#ridge)    
    - [Decision Tree](#tree)
    - [K-nn](#knn)

    

## Ground Truths About Data <a class="anchor" id="facts"></a>

 - seasons column's values stand for: 
1. spring
2. summer
3. fall
4. winter 


 - weather stands for: 
1. Clear, Few clouds, Partly cloudy, Partly cloudy
2. Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
3. Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
4. Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 


 - temp - temperature in Celsius
 - atemp - "feels like" temperature in Celsius
 - casual - number of non-registered user rentals initiated
 - registered - number of registered user rentals initiated
 - count - number of total rentals



***

# EDA and Feature Engineering <a class="anchor" id="EDA"></a>

### Importing packages and reading the data <a class="anchor" id="import"></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import math
import warnings
warnings.filterwarnings("ignore")


In [ ]:
#reading data
data = pd.read_csv("../input/bike-sharing-demand/train.csv")
data

In [ ]:
data.info()

<div class="alert alert-block alert-info">

the datetime column is type object, so it needs some engineering
</div>

In [ ]:
#reformatting the datetime feature
data['datetime'] = pd.to_datetime(data['datetime'],format='%Y-%m-%d %H:%M:%S')

#add new columns from the datetime column
data['month'] = data['datetime'].dt.month 
data['day'] = data['datetime'].dt.day
data['hour'] = data['datetime'].dt.hour 
data['dayofweek'] = data['datetime'].dt.dayofweek 

data = data.drop(["datetime"],axis=1)

In [ ]:
data.describe()

- it appears that there's no nulls or missing values in the dataframe
- it appears that there's some outliers in the (casual, registered and count) columns since the max value is a lot higher than the 75%(quartile) value
- the rest of the columns appear to be normally distributed


### Log Transform  <a class="anchor" id="log"></a>
#### Let's look at the outliers in casual, registered and count using frequency plots

In [ ]:
fig, ax = plt.subplots(3,2, figsize=(16,15))

#casual customers distribution plots
sns.distplot(data["casual"],ax=ax[0][0])
data.boxplot("casual", ax=ax[0][1])

#registered customers distribution plots
sns.distplot(data["registered"], ax=ax[1][0])
data.boxplot("registered", ax=ax[1][1])

#all customers distribution plots
sns.distplot(data["count"], ax=ax[2][0])
data.boxplot("count", ax=ax[2][1])

fig.show()

<div class="alert alert-block alert-info">
it's obvious that the three columns are positively skewed, let's use <b>log transform </b> to normalize the distribution a bit.
</div>

In [ ]:
#normalizing columns(casual, registered and count) using log transform
data['casual_log'] = np.log((1+ data['casual']))
data['registered_log'] = np.log((1+ data['registered']))
data['count_log'] = np.log((1+ data['count']))

#resulting distributing
fig, ax = plt.subplots(1,3, figsize=(16,5))
data.boxplot("casual_log", ax=ax[0])
data.boxplot("registered_log", ax=ax[1])
data.boxplot("count_log", ax=ax[2])

fig.show()

<div class="alert alert-block alert-success">
these almost normally distributed values will be used in the ML models later in this notebook. 
</div>

## Distributions of the bookings according to the features: <a class="anchor" id="features"></a>
 - season 
 - month
 - day
 - day of the week 
 - hour
 - weather 
 - windspeed
 - temp
 - humidity

and see how the bookings are distributed into casual and registered for each category

<div class="alert alert-block alert-info">
First let's look at the correlation matrix to get an overview of how the variable are connected
</div>

In [ ]:
corr = data.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

f, ax = plt.subplots(figsize=(11, 9))

# Draw the heatmap
sns.heatmap(corr, mask=mask, center=0,
            square=True, linewidths=.5)


<div class="alert alert-block alert-success">
you can see that there's some <b>slight correlation</b> between some values, but the correlation for each feature is too small to make the prediction dependent on one value, which is why a machine learning model is needed.
</div>

In [ ]:
def plot_category_with_user_type(category): 
    ''' 
    function to plot inputed category with its count, casual and registered distributions
    input: category name
    output: showing 3 plots for count, casual and registered respectively
    '''
    #set plot up
    fig, ax = plt.subplots(1,3, figsize=(16,5), sharey=True)
    #title
    fig.suptitle("User Distribution According To " + category, fontsize=14)
    #plot 3 plots for count,casual and registered customers 
    sns.barplot(x = category, y = "count", data = data, ax = ax[0])
    sns.barplot(x = category, y = "casual", data = data, ax = ax[1])
    sns.barplot(x = category, y = "registered", data = data, ax = ax[2])
    fig.show()

### By season

In [ ]:
plot_category_with_user_type("season")

### By Month

In [ ]:
plot_category_with_user_type("month")

users (registered and casual) show a <b>pattern through the seasons</b>, bookings are at their lowest in the spring (season1), they peak in summer and fall(season2+3), and fall a bit in winter(season4).<br>
matching results show in the months distribution.


## By Day <a class="anchor" id="months"></a>

In [ ]:
plot_category_with_user_type("day")

### The distribution is <b>consistent throughout the month</b> 

### - By Day Of Week <a class="anchor" id="weeks"></a>

In [ ]:
plot_category_with_user_type("dayofweek")


 - The number of bookings is consistent in general throughout the week for the company, but there's a slight fall on sunday which is a holiday.<br> 
 - There's a pattern among casual non-registering users, there's a peak of usage on saturday and sunday, probably because saturday is the end of the week which means high traffic, and friday is the weekend. 
 - There's a pattern among registered users, the booking are consistent throughout the week except fot saturday and sunday, which can be the result of multiple reasons that need to be considered in later research : 
     1. There isn't enough bikes because of the higher demand on bikes by non-registered users near weekends which means more bikes should be introduced to the system.
     2. Regular users might not be enjoying the experience so much which can result on only using the bikes when flexability and speed are needed rather for leisure.

### - By Hour

In [ ]:
plot_category_with_user_type("hour")

 - casual users usually use bikes in the middle of the day and consistently peak from 12-6pm
 - registered users bookings peak at 8am and 5-6pm which strenghthens the theory that they use bikes for work rather than leisure.

### - By Weather

In [ ]:
plot_category_with_user_type("weather")

 - casual users tend to book more when the weather is better
 - registered users surprisingly book consistently except when the weather is "Light Snow, light rain..."(number3)
 
 the difference in patterns strengthenes the theory of different purposes between registered and casual users
 

### - By Windspeed

In [ ]:
plot_category_with_user_type("windspeed")

the distribution is almost consistent regardless of the windspeed except for a couple of outliers.<br>
Windspeed should be excluded.

### - By Temp


In [ ]:
plot_category_with_user_type("temp")

 there's a clear connection between the tempreture and the increasing bookings.

### - By Humidity

In [ ]:
plot_category_with_user_type("humidity")

people tend to rent bikes when the humidity is lower than usual, but the change isn't that drastic and the variance is big

### Distribution of booking by Hours and Day-of-Week Together <a class="anchor" id="dayhour"></a>

In [ ]:
#setting plot up
fig, ax = plt.subplots(1,7, figsize=(50,7), sharey=True)

days=["Monday", "Tuesday", "Wednsdays", "Thursday", "Friday","Saturday", "Sunday"]

for i in range(7): 
    #split data according to week-day
    day_data= data[data["dayofweek"] == i]
    #plot day's dstribution
    sns.barplot(x = "hour", y = "count", data = day_data, ax = ax[i]).set_title("Booking Distribution by Hours on " + days[i])
fig.show()

the distribution is almost consistent in workdays and peaks on start and end times of typical jobs, but it changes in the weekend and becomes distributed throughout the day peaking in the middle of the day.

## Weather-Related values Collerence  <a class="anchor" id="weather"></a>
### Let's take a look at "atemp" and answer these questions
- is atemp always bigger than the real tempreture?
- is there a connection between atemp and humidity? if the answer to the first question is yes can the reason be humidity?
- should we use it in the models?


In [ ]:
#calculate the difference between temp and atemp
averege_diff_atemp_and_temp = data["atemp"] - data["temp"]
#binarize the resulting values to classify as bigger or smaller
averege_diff_atemp_and_temp[averege_diff_atemp_and_temp >= 0] = 1
averege_diff_atemp_and_temp[averege_diff_atemp_and_temp < 0] = 0
#sum the result to calculate the precentage of atemps higher than temps
sum(averege_diff_atemp_and_temp) / len(averege_diff_atemp_and_temp)

<div class="alert alert-block alert-success">
    so yes,<b> 99%</b> of the times atemp is <b>greater</b> than the real tempreture!
    </div>

In [ ]:
#calculate the correlation between temp, atemp and humidity
correlation = data[["temp", "atemp", "humidity"]].corr()
correlation

In [ ]:
fig = plt.figure(figsize=(10,5))
plt1 = fig.add_subplot(121)
plt2 = fig.add_subplot(122)

#naming plots
plt1.set_title('atemp vs temp')
plt2.set_title('atemp vs hyumidiy')
#ploting plots
plt1.scatter(data["atemp"], data["temp"])
plt2.scatter(data["atemp"], data["humidity"])



by looking at the correlation values and the scatter plot, atemp and temp are clearly correlated while atemp and humidity not correlated at all.
   <div class="alert alert-block alert-success">
 
 since atemp is highly correlated with temp with a 99% rate, <b>atemp won't be used in building the models</b> and will be dropped.
    </div>

In [ ]:
#dropping atemp
data = data.drop(["atemp"],axis=1)

### Is there a correlence between the season, temperature, weather, windspeed and the humidity?

In [ ]:
#calculating correlence array for the attributes bellow
correlation = data[["season","temp", "weather", "windspeed", "humidity"]].corr()
correlation

In [ ]:
plt.scatter(data["weather"], data["humidity"])

<div class="alert alert-block alert-info">
as you can see there's no significant correlation between the weather, temperature, weather, windspeed and humidity, so all features will be used in the model
</div>

### Is the workingday atribute enough? (deleting day, dayofweek and holiday) <a class="anchor" id="workingday"></a>

<div class="alert alert-block alert-info">
    <b>let's look at the distibution of bookings in holidays, is it similar to regular days?</b>    
</div>

In [ ]:
#plotting the holidays distribution excluding the weekends 
holiday_data = data[(data["holiday"]==1) & (data["dayofweek"]<5)]

sns.barplot(x = "hour", y = "count", data = holiday_data)

as seen above, The booking distribution in holidays is very similar to [weekends](#dayhour), and the workingday bolean atribute is taking this variable in account (it's 1 when the day isn't a holiday or a weekend).<br>
<div class="alert alert-block alert-success">
    <b>there's no need for the holiday attribute</b>
</div>

[We've previously seen](#months) that the distribution of bookings is consistent throughout the month regardles of the days number which leads us to believe that:
<div class="alert alert-block alert-success">
    <b>there's no need for the day attribute</b>
</div>

[We've also seen](#weeks) that the distribution among weeks is consistent, and is about the same for all days except for weekends. which is considered in the workday attribute, leads us to believe that: 
<div class="alert alert-block alert-success">
    <b>there's no need for the dayofweek attribute</b>
</div>

### Hot-Encoding the Weather and Season attributes  <a class="anchor" id="hot"></a>

<div class="alert alert-block alert-info">
Weather and Season attributes are categorical, which makes them unacceptable for a lot of models, we will use Hot-encoding to turn these categorical values into binary valueS for each label
    </div>

In [ ]:
hot_encoded_data = data.join(pd.get_dummies(data["season"], prefix="season")).join(pd.get_dummies(data["weather"], prefix="weather"))
hot_encoded_data.head()

### Dropping Useless Columns <a class="anchor" id="drop">

In [ ]:
hot_encoded_data = hot_encoded_data.drop(["count", "registered","casual"],axis=1)
hot_encoded_data = hot_encoded_data.drop(["holiday", "dayofweek"],axis=1)
hot_encoded_data = hot_encoded_data.drop(["month", "season", "weather", "windspeed"],axis=1)

*** 
# Machine Learning Models <a class="anchor" id="ML">

In the next section we will be trying out the following machine learning models: 
 - Linear Regression
 - Linear Regression - polynomial features
 - Ridge
 - Descion tree
 - K-nn

In [ ]:
#importing packages
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

In [ ]:
def partition_data(full_data):
    ''' 
    takes a dataframe and seperates it into train and test data by splitting it by "day" 
    input: full dataframe
    output: x-axis training set
    output: y-axis training set
    output: x-axis test set
    output: x-axis test set    
    '''
    #split data by day
    train_data = full_data[data["day"]<15]
    test_data = full_data[data["day"]>14]

    #dropping the day column
    train_data = train_data.drop(["day"],axis=1)
    test_data = test_data.drop(["day"],axis=1)
    
    #splitting the training data into x and y axises
    x_train = train_data.drop(["casual_log", "count_log", "registered_log"],axis=1)
    y_train = train_data[["count_log"]]

    #splitting the test data into x and y axises
    x_test = test_data.drop(["casual_log", "count_log", "registered_log"],axis=1)
    y_test = test_data[["count_log"]]
    
    return x_train, y_train, x_test, y_test

In [ ]:
def build_model(data, model, plot = False, tuning_parameter = 1, result= False):
    '''
    build and return model and MSE according to the entered keyword
    input: full dataframe
    input: model keyword
    input: boolean value to draw a plot or not -optional-
    input: tuning parameter for the model
    output: the model
    output: MSE value
    '''
    #split data using the previous function
    x_train, y_train, x_test, y_test = partition_data(data)
    
    #models dictionary
    models={
        "linear" : LinearRegression(),
        "ridge" : Ridge(alpha = tuning_parameter),
        "knn" : KNeighborsRegressor(n_neighbors = tuning_parameter),
        "decision_tree" : DecisionTreeRegressor(random_state=42)
    }
    
    #initiate model
    lModel = models[model]
    #fit the training data in the model
    lModel.fit(X = x_train, y = y_train)
    
    #predict new values
    y_predicted=lModel.predict(X= x_test) 

    #calculate MSE
    MSE = mean_squared_error(y_test, y_predicted)
    
    #plot real vs predicted values if the plot boolean is true
    if (plot):
        plt.figure(figsize=(5, 5))
        plt.scatter(y_test, y_predicted)
        plt.ylabel("predicted values")
        plt.xlabel("real values")
        #find min and max in the test target values
        minimum = math.ceil(y_test.min())
        maximum = math.ceil(y_test.max())
        #plot a diagonal line accross the scattered plot to better see the difference in values
        #idealy all points should be on the diagonal line
        plt.plot( [minimum,maximum],[minimum,maximum], color='red')
    if (result): return lModel, MSE, y_predicted
    return lModel, MSE

In [ ]:
def undo_log_transform(data): 
    return (np.floor(np.exp(data) -1))


## Linear Regression <a class="anchor" id="linear">

In [ ]:
#build a linear model and plot the result
model, MSE, linear_result = build_model(hot_encoded_data, "linear", plot = True, result = True)

In [ ]:
#vlaues predicted from model
undo_log_transform(linear_result)

<div class="alert alert-block alert-info">
Let's try to use polynomial features to improve the model
    </div>

## Linear Regression-polynomial features <a class="anchor" id="linearp">

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

#set up the polynomial fearues object
polynomial_fearues = PolynomialFeatures(degree=4, interaction_only=True)

polynomial_fearures_data = hot_encoded_data[["workingday", "hour", "temp", "humidity"]]

#make polynomial features out of the data
polynomial_fearues = polynomial_fearues.fit_transform(polynomial_fearures_data)
#turn the result into a dataframe
intr_features = pd.DataFrame(polynomial_fearues)

In [ ]:
list_of_MSEs = []
list_of_thetas = []

#for each column in the polynomial fearues dataset: append it to the data and test the model
for (columnName, columnData) in intr_features.iteritems():
    polynomial_hypothesis_data = hot_encoded_data.drop(["workingday", "hour", "temp", "humidity"],axis=1)
    #appending the column
    polynomial_hypothesis_data["polynomial_factor"] = columnData
    
    #build model and assess it
    model, MSE = build_model(polynomial_hypothesis_data, "linear")
    
    list_of_MSEs.append(MSE)
    list_of_thetas.append(model.coef_)
    
list_of_MSEs

<div class="alert alert-block alert-info">
The MSEs values are bigger when using polynomial features than when using simple linear regression, so we will ignore them.
   </div>

## Ridge <a class="anchor" id="ridge">

In [ ]:
#partition data to find the best tunning parameter
x_train, y_train, x_test, y_test = partition_data(hot_encoded_data)

#this object can find the best alpha to be used for this model
regr_cv = RidgeCV(alphas=[0.1, 1.0, 5.0, 10.0, 20, 25, 30, 40, 50, 100, 200, 500, 1000, 2000])

model_cv = regr_cv.fit(x_train, y_train)
model_cv.alpha_

In [ ]:
#build ridge model with the found alpha
model, MSE, ridge_result = build_model(hot_encoded_data, "ridge", tuning_parameter = 25, plot = True, result = True)
MSE

In [ ]:
#vlaues predicted from model
undo_log_transform(ridge_result)

## Decision Tree  <a class="anchor" id="tree">

In [ ]:
#prepare data for the model
tree_data = data.drop(["month", "count", "registered","casual","holiday", "dayofweek", "windspeed"],axis=1)
#build and assess model
model, MSE, tree_result =build_model(tree_data, "decision_tree", plot = True, result = True)
MSE

In [ ]:
#vlaues predicted from model
undo_log_transform(tree_result)

In [ ]:
#packages for visuallizing the decision tree
from sklearn import tree
import graphviz
from IPython.display import SVG
from IPython.display import display

In [ ]:
#find columns names
columns = tree_data.drop(["casual_log", "count_log", "registered_log", "day"],axis=1).columns

#plot each feature with its importance in the model
plt.figure(figsize=(10, 5))
plt.bar(columns, model.feature_importances_)
plt.ylabel("importance")
plt.xlabel("features")


In [ ]:
#plotting the decision tree
tree_data = tree.export_graphviz(model, feature_names = columns, filled=True, rounded=True,max_depth=6)

graph = graphviz.Source(tree_data)
display(SVG(graph.pipe(format='svg')))

## K-NN  <a class="anchor" id="knn">

In [ ]:
#scalling the target data for the K-nn model
scaler = StandardScaler()
general_data = hot_encoded_data[["casual_log", "count_log", "registered_log"]]
scaler.fit(general_data)
scaled = scaler.transform(general_data)
scaled = pd.DataFrame(data=scaled, columns = general_data.columns)

#turning the target log value into int because K-nn doesn't work with float values
scaled["casual_log"] = (scaled["casual_log"] * 10) .apply(np.floor)
scaled["registered_log"] = (scaled["registered_log"] * 10) .apply(np.floor)
scaled["count_log"] = (scaled["count_log"] * 10) .apply(np.floor)

#join the new columns
general_data = general_data.join(hot_encoded_data.drop(["casual_log", "count_log", "registered_log"],axis=1))

In [ ]:
#finding the best k value
k_values = [3, 4, 5, 6, 7, 8, 9, 10, 15, 30]

MSEs_values =[]
models = []

for k_value in k_values: 
    #build and assess model
    model, MSE = build_model(general_data, "knn", tuning_parameter = k_value)
    MSEs_values.append(MSE)
    models.append(model)
MSEs_values

<b>7</b> is the best value for k 

In [ ]:
model, MSE, knn_result = build_model(general_data, "knn", tuning_parameter = 7, plot = True, result = True)

In [ ]:
#vlaues predicted from model
undo_log_transform(knn_result)